In [1]:
import pandas as pd

In [2]:
def movies_genres_insert(movie_names,genres_names):
    queries = ""
    for movie_name in movie_names:
        for genres_name in genres_names:
            query = f"""INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = '{movie_name}' AND g.name = '{genres_name}';

"""
            queries = queries + query
    return queries

In [3]:
print(movies_genres_insert(['Тайна Коко'],['Мультфильмы','Фэнтези','Комедия','Приключения','Семейные','Музыкальные']))

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Тайна Коко' AND g.name = 'Мультфильмы';

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Тайна Коко' AND g.name = 'Фэнтези';

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Тайна Коко' AND g.name = 'Комедия';

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Тайна Коко' AND g.name = 'Приключения';

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Тайна Коко' AND g.name = 'Семейные';

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Та

In [4]:
def films_positions_insert(member_names,movie_names,positions_names):
    queries_0 = """DO $$"""
    for i in range(len(member_names)):
        query = f"""
DECLARE id_member_{i} INT;
DECLARE id_movie_{i} INT;"""
        queries_0 = queries_0 + query
        
    queries_1 = """
BEGIN"""
    for i in range(len(member_names)):
        i_names = [j for j, name in enumerate(member_names[:i]) if name == member_names[i]]
        if len(i_names) > 0:
            query_1 = f"""
	id_member_{i} = id_member_{i_names[0]};
"""
        else:
            query_1 = f"""
	SELECT cm.id INTO id_member_{i}
	FROM "public.Film_crew_members" cm
	WHERE cm.name = '{member_names[i]}';
"""
        i_movies = [j for j, name in enumerate(movie_names[:i]) if name == movie_names[i]]
        if len(i_movies) > 0:
            query_2 = f"""
	id_movie_{i} = id_movie_{i_movies[0]};
"""
        else:
            query_2 = f"""
	SELECT m.id INTO id_movie_{i}
	FROM "public.Movies" m
	WHERE m.name = '{movie_names[i]}';
    """
        queries_1 = queries_1 + query_1 + query_2

    queries_2 = f"""
	INSERT INTO "public.Films_positions" (member_id, movie_id, name) VALUES"""
    for i in range(len(member_names)):
        query = f"""
		(id_member_{i}, id_movie_{i}, '{positions_names[i]}')"""
        if i == len(member_names)-1: query_end = """;"""
        else: query_end = ""","""
        queries_2 = queries_2 + query + query_end
        
    queries_3 = """
END $$;"""
        
    return queries_0 + queries_1 + queries_2 + queries_3

In [5]:
excel_file = 'sql_movie_positions.xlsx'

df_movie_positions = pd.read_excel(excel_file)

print(df_movie_positions)

                   Люди                               Фильмы Должности
0             Том Хэнкс                         Зеленая миля     Актер
1             Том Хэнкс                         Форрест Гамп     Актер
2           Тим Роббинс                    Побег из Шоушенка     Актер
3         Морган Фриман                    Побег из Шоушенка     Актер
4         Франсуа Клюзе                                  1+1     Актер
5               Омар Си                                  1+1     Актер
6            Лиам Нисон                      Список Шиндлера     Актер
7      Мэттью МакКонахи                         Интерстеллар     Актер
8           Энн Хэтэуэй                         Интерстеллар     Актер
9           Элайджа Вуд  Властелин колец: Возвращение короля     Актер
10         Орландо Блум  Властелин колец: Возвращение короля     Актер
11        Эдвард Нортон                      Бойцовский клуб     Актер
12            Брэд Питт                      Бойцовский клуб     Актер
13  Хе

In [6]:
member_names = df_movie_positions['Люди']
movie_names = df_movie_positions['Фильмы']
positions_names = df_movie_positions['Должности']
print(films_positions_insert(member_names,movie_names,positions_names))

DO $$
DECLARE id_member_0 INT;
DECLARE id_movie_0 INT;
DECLARE id_member_1 INT;
DECLARE id_movie_1 INT;
DECLARE id_member_2 INT;
DECLARE id_movie_2 INT;
DECLARE id_member_3 INT;
DECLARE id_movie_3 INT;
DECLARE id_member_4 INT;
DECLARE id_movie_4 INT;
DECLARE id_member_5 INT;
DECLARE id_movie_5 INT;
DECLARE id_member_6 INT;
DECLARE id_movie_6 INT;
DECLARE id_member_7 INT;
DECLARE id_movie_7 INT;
DECLARE id_member_8 INT;
DECLARE id_movie_8 INT;
DECLARE id_member_9 INT;
DECLARE id_movie_9 INT;
DECLARE id_member_10 INT;
DECLARE id_movie_10 INT;
DECLARE id_member_11 INT;
DECLARE id_movie_11 INT;
DECLARE id_member_12 INT;
DECLARE id_movie_12 INT;
DECLARE id_member_13 INT;
DECLARE id_movie_13 INT;
BEGIN
	SELECT cm.id INTO id_member_0
	FROM "public.Film_crew_members" cm
	WHERE cm.name = 'Том Хэнкс';

	SELECT m.id INTO id_movie_0
	FROM "public.Movies" m
	WHERE m.name = 'Зеленая миля';
    
	id_member_1 = id_member_0;

	SELECT m.id INTO id_movie_1
	FROM "public.Movies" m
	WHERE m.name = 'Форрест

In [7]:
def nominees_insert(member_names,movie_names):
    queries_0 = """DO $$"""
    for i in range(len(member_names)):
        query_1 = ""; query_2 = ""
        if len(member_names[i]) > 0:
            query_1 = f"""
DECLARE id_member_{i} INT;"""
        if len(movie_names[i]) > 0:
            query_2 = f"""
DECLARE id_movie_{i} INT;"""
        queries_0 = queries_0 + query_1 + query_2
        
    queries_1 = """
BEGIN"""
    for i in range(len(member_names)):
        query_1 = ""; query_2 = ""
        if len(member_names[i]) > 0:
            i_names = [j for j in range(i) if member_names[j] == member_names[i]]
            if len(i_names) > 0:
                query_1 = f"""
	id_member_{i} = id_member_{i_names[0]};
"""
            else: #если нет повторяющихся людей
                query_1 = f"""
	SELECT cm.id INTO id_member_{i}
	FROM "public.Film_crew_members" cm
	WHERE cm.name = '{member_names[i]}';
"""
        if len(movie_names[i]) > 0:
            i_movies = [j for j in range(i) if movie_names[j] == movie_names[i]]
            if len(i_movies) > 0:
                query_2 = f"""
	id_movie_{i} = id_movie_{i_movies[0]};
"""
            else: #если нет повторяющихся фильмов
                query_2 = f"""
	SELECT m.id INTO id_movie_{i}
	FROM "public.Movies" m
	WHERE m.name = '{movie_names[i]}';
    """
        queries_1 = queries_1 + query_1 + query_2

    queries_2 = f"""
	INSERT INTO "public.Nominees" (film_crew_members_id, movie_id) VALUES"""
    for i in range(len(member_names)):
        query = ""
        if (len(member_names[i]) > 0) & (len(movie_names[i]) > 0):
            query = f"""
		(id_member_{i}, id_movie_{i})"""
        elif (len(member_names[i]) > 0) & (len(movie_names[i]) == 0):
            query = f"""
		(id_member_{i}, null)"""
        elif (len(member_names[i]) == 0) & (len(movie_names[i]) > 0):
            query = f"""
		(null, id_movie_{i})"""
        if i == len(member_names)-1: query_end = """;"""
        else: query_end = ""","""
        queries_2 = queries_2 + query + query_end
        
    queries_3 = """
END $$;"""
        
    return queries_0 + queries_1 + queries_2 + queries_3

In [8]:
excel_file = 'sql_nominees.xlsx'

df_nominees = pd.read_excel(excel_file)
df_nominees = df_nominees.fillna('')

print(df_nominees)

        Люди        Фильмы
0  Том Хэнкс  Зеленая миля
1  Том Хэнкс  Форрест Гамп
2             Зеленая миля
3             Форрест Гамп


In [9]:
member_names = df_nominees['Люди']
movie_names = df_nominees['Фильмы']
print(nominees_insert(member_names,movie_names))

DO $$
DECLARE id_member_0 INT;
DECLARE id_movie_0 INT;
DECLARE id_member_1 INT;
DECLARE id_movie_1 INT;
DECLARE id_movie_2 INT;
DECLARE id_movie_3 INT;
BEGIN
	SELECT cm.id INTO id_member_0
	FROM "public.Film_crew_members" cm
	WHERE cm.name = 'Том Хэнкс';

	SELECT m.id INTO id_movie_0
	FROM "public.Movies" m
	WHERE m.name = 'Зеленая миля';
    
	id_member_1 = id_member_0;

	SELECT m.id INTO id_movie_1
	FROM "public.Movies" m
	WHERE m.name = 'Форрест Гамп';
    
	id_movie_2 = id_movie_0;

	id_movie_3 = id_movie_1;

	INSERT INTO "public.Nominees" (film_crew_members_id, movie_id) VALUES
		(id_member_0, id_movie_0),
		(id_member_1, id_movie_1),
		(null, id_movie_2),
		(null, id_movie_3);
END $$;


In [10]:
def prime_nominations_insert(member_names,movie_names,prime_names,nominations,years,isWon):
    queries_0 = """DO $$
"""
    for i in range(len(member_names)):
        query = f"""DECLARE id_nominee_{i} INT;
DECLARE id_prime_{i} INT;
"""
        queries_0 = queries_0 + query
        
    queries_1 = """
BEGIN"""
    for i in range(len(member_names)):
        query_1 = ""; query_2 = ""; query_3 = ""; query_4 = ""; query_5 = ""
        i_members_movies = [j for j in range(i) if (member_names[j] == member_names[i]) & (movie_names[j] == movie_names[i])]
        if len(i_members_movies) == 0: #если нет повторяющихся номинантов
            query_1 = f"""
	SELECT nom.id INTO id_nominee_{i}
	FROM "public.Nominees" nom
	JOIN "public.Movies" m ON nom.movie_id = m.id"""
            if len(member_names[i]) > 0:
                query_2 = f"""
	JOIN "public.Film_crew_members" cm ON nom.film_crew_members_id = cm.id"""
            query_3 = f"""
	WHERE m.name = '{movie_names[i]}' AND """
            if len(member_names[i]) > 0:
                query_4 = f"""cm.name = '{member_names[i]}';
"""
            else: query_4 = f"""nom.film_crew_members_id IS NULL;
"""
        else: query_1 = f"""
	id_nominee_{i} = id_nominee_{i_members_movies[0]};
"""
        i_primes = [j for j in range(i) if prime_names[j] == prime_names[i]]
        if len(i_primes) == 0: #если нет повторяющихся премий
            query_5 = f"""
	SELECT p.id INTO id_prime_{i}
	FROM "public.Primes" p
	WHERE p.name = '{prime_names[i]}';
    """
        else: query_5 = f"""
	id_prime_{i} = id_prime_{i_primes[0]};
"""
        queries_1 = queries_1 + query_1 + query_2 + query_3 + query_4 + query_5

    queries_2 = f"""
	INSERT INTO "public.Prime_nominations" (nominee_id, prime_id, nomination, year, "isWon") VALUES"""
    for i in range(len(member_names)):
        query = f"""
		(id_nominee_{i}, id_prime_{i}, '{nominations[i]}', {int(years[i])}, {isWon[i]})"""
        if i == len(member_names)-1: query_end = """;"""
        else: query_end = ""","""
        queries_2 = queries_2 + query + query_end
        
    queries_3 = """
END $$;"""
        
    return queries_0 + queries_1 + queries_2 + queries_3

In [11]:
excel_file = 'sql_prime_nominations.xlsx'

df_prime_nominations = pd.read_excel(excel_file)
df_prime_nominations = df_prime_nominations.fillna('')

print(df_prime_nominations)

         Люди        Фильмы                  Премии  \
0   Том Хэнкс  Зеленая миля  Премия Гильдии актеров   
1   Том Хэнкс  Форрест Гамп                   Оскар   
2   Том Хэнкс  Форрест Гамп          Золотой глобус   
3   Том Хэнкс  Форрест Гамп     Премия канала «MTV»   
4   Том Хэнкс  Форрест Гамп     Британская академия   
5   Том Хэнкс  Форрест Гамп                  Сатурн   
6   Том Хэнкс  Форрест Гамп  Премия Гильдии актеров   
7              Зеленая миля                   Оскар   
8              Зеленая миля                  Сатурн   
9              Зеленая миля                   Оскар   
10             Форрест Гамп                   Оскар   
11             Форрест Гамп          Золотой глобус   
12             Форрест Гамп     Премия канала «MTV»   
13             Форрест Гамп     Британская академия   

                                           Номинации     Год  Статус  
0                            Лучший актерский состав  2000.0   False  
1                               

In [12]:
member_names = df_prime_nominations['Люди']
movie_names = df_prime_nominations['Фильмы']
prime_names = df_prime_nominations['Премии']
nominations = df_prime_nominations['Номинации']
years = df_prime_nominations['Год']
isWon = df_prime_nominations['Статус']

print(prime_nominations_insert(member_names,movie_names,prime_names,nominations,years,isWon))

DO $$
DECLARE id_nominee_0 INT;
DECLARE id_prime_0 INT;
DECLARE id_nominee_1 INT;
DECLARE id_prime_1 INT;
DECLARE id_nominee_2 INT;
DECLARE id_prime_2 INT;
DECLARE id_nominee_3 INT;
DECLARE id_prime_3 INT;
DECLARE id_nominee_4 INT;
DECLARE id_prime_4 INT;
DECLARE id_nominee_5 INT;
DECLARE id_prime_5 INT;
DECLARE id_nominee_6 INT;
DECLARE id_prime_6 INT;
DECLARE id_nominee_7 INT;
DECLARE id_prime_7 INT;
DECLARE id_nominee_8 INT;
DECLARE id_prime_8 INT;
DECLARE id_nominee_9 INT;
DECLARE id_prime_9 INT;
DECLARE id_nominee_10 INT;
DECLARE id_prime_10 INT;
DECLARE id_nominee_11 INT;
DECLARE id_prime_11 INT;
DECLARE id_nominee_12 INT;
DECLARE id_prime_12 INT;
DECLARE id_nominee_13 INT;
DECLARE id_prime_13 INT;

BEGIN
	SELECT nom.id INTO id_nominee_0
	FROM "public.Nominees" nom
	JOIN "public.Movies" m ON nom.movie_id = m.id
	JOIN "public.Film_crew_members" cm ON nom.film_crew_members_id = cm.id
	WHERE m.name = 'Зеленая миля' AND cm.name = 'Том Хэнкс';

	SELECT p.id INTO id_prime_0
	FROM "publ